In [3]:
:set -XOverloadedLabels -XOverloadedLists
import Topoll.Samplers.Samplers
import Topoll.DistanceMatrix.DistanceMatrix
import Topoll.Complexes.Witness
import Topoll.Complexes.Rips
import Topoll.SimplicialSet
import Topoll.ChainComplex.Type
import Topoll.Drawings
import Named
import qualified Data.Vector as V


In [4]:
pts <- sampleSphereUniformly ! #r 10 ! #n 100
-- pts <- sampleTorusUniformly ! #r 2 ! #bigR 10 ! #n 100

-- dat <- chooseMaxminLandmarks 12 pts
dat <- chooseRandomLandmarks 12 pts

wComplex nu r = case witnessSet (computeDistanceMatrix dat) ! #nu nu ! #r r of
  Right cc -> cc
  Left err -> error err

rComplex r = ripsSet (computeTotalDistanceMatrix (landmarkPoints dat)) ! #r r


In [ ]:
printComplexAndHomology r = do
  putStrLn $ "r = " <> show r
  -- let complex = rComplex r
  let complex = wComplex 2 r
  case simplicialComplex complex of
    SomeChainComplex chainComplex -> putStrLn $ reprChainComplexModules chainComplex
  print . simplicialHomologyOverQ $ complex
  putStrLn ""

mapM_ printComplexAndHomology [0,0.5 .. 4]


In [12]:
vlShow $ drawPersistentHomologies (simplicialHomologyOverQ . rComplex) eqTorus 0.25 40


In [25]:
import Data.List (intercalate)
import Text.Printf
toWolfram :: V.Vector Point -> IO ()
toWolfram points = do
  let f = printf "%.3f"
      pointsInBraces =
        "{" <> intercalate ","
            (map (\[x,y,z] -> "{" <> f x <> "," <> f y <> ", " <> f z <> "}")
             (V.toList points)) <> "}"
  putStrLn pointsInBraces
toWolfram pts


{{-3.264,9.886, 1.957},{8.865,-5.442, 1.959},{10.536,-3.547, 1.659},{-8.636,5.164, 1.999},{-3.096,-7.383, -0.156},{10.284,-4.643, 1.534},{-4.529,6.595, -0.031},{-4.254,8.568, -1.952},{8.417,7.689, -1.428},{3.545,-9.579, 1.989},{-11.109,4.534, -0.059},{3.286,7.775, 1.253},{11.423,3.570, 0.359},{3.168,11.567, 0.167},{9.509,3.646, -1.992},{10.600,5.143, -0.908},{10.657,3.264, -1.640},{5.058,-9.774, 1.729},{6.218,-6.302, 1.639},{-0.872,8.031, 0.554},{2.721,9.510, 1.997},{4.983,7.325, -1.643},{7.499,-3.818, 1.220},{-7.652,7.671, 1.817},{10.254,-6.133, 0.453},{-8.408,-7.605, -1.487},{-10.203,-2.305, -1.946},{-6.166,10.237, -0.440},{6.987,4.422, 1.002},{-6.961,-3.946, -0.076},{10.618,1.182, 1.880},{-4.652,-8.217, -1.921},{-5.168,8.481, 1.999},{2.008,11.467, -1.142},{-9.030,3.499, -1.975},{0.547,-7.998, -0.258},{-5.184,7.164, -1.631},{-4.843,-7.531, 1.705},{7.798,2.318, 0.724},{-5.882,6.077, -1.273},{10.927,-2.274, 1.628},{10.687,4.065, -1.394},{1.181,11.030, 1.675},{-11.451,3.336, 0.536},{9.0